In [5]:
"""
COMPREHENSIVE ANNOTATION:
Cell Patch Extraction Pipeline for Xenium Spatial Transcriptomics Data

This code performs extraction and quality assessment of cell patches from H&E (Hematoxylin and Eosin) 
stained histology images, aligned with spatial transcriptomics data from the Xenium platform. 
The goal is to extract image patches centered around cell nuclei detected in Xenium data.

Key functionalities:
1. Loading spatial transcriptomics data and corresponding H&E images
2. Converting nucleus coordinates from microns to pixels
3. Sampling cells in a spatially distributed manner
4. Extracting image patches at multiple zoom levels (kappa values)
5. Evaluating the quality of extracted patches
6. Selecting the best patches based on quality metrics

The pipeline serves as a preprocessing step for downstream analysis that combines 
histological features with gene expression data at single-cell resolution.
"""

import os
import time
import random
import xml.etree.ElementTree as ET  # For parsing XML metadata in OME-TIFF files

# Numerical and data processing libraries
import numpy as np  # For efficient numerical operations
import pandas as pd  # For data manipulation and analysis
import matplotlib.pyplot as plt  # For visualization

# Image processing libraries
from scipy import ndimage  # For image transformations like distance transforms
from skimage import exposure, filters, measure, color, transform  # For advanced image processing
from tifffile import TiffFile  # For reading TIFF files with metadata
import spatialdata as sd  # Library for spatial transcriptomics data

from tqdm.notebook import tqdm  # Progress bar for Jupyter notebooks

# =============================================================================
# Module-level cache for heavy data
# =============================================================================
# Why use module-level caching? 
# Answer: Loading large spatial datasets and high-resolution images is computationally expensive.
# Caching these objects prevents redundant loading operations, significantly improving performance
# when functions are called multiple times.
# _cached_sdata = None  # Cache for SpatialData object
# _cached_he_image = None  # Cache for H&E image
# _cached_pixel_size = None  # Cache for pixel size (psx, psy) in microns

# Paths to data files
# Why hardcode paths?
# Answer: While not ideal for production, hardcoding simplifies development and testing.
# In a production environment, these would be parameterized or loaded from configuration.
base_dir      = "/Users/jianzhouyao/Cancer"
xenium_dir    = os.path.join(base_dir, "Pancreatic Cancer with Xenium Human Multi-Tissue and Cancer Panel")
zarr_path     = os.path.join(xenium_dir, "Xenium_V1_hPancreas_Cancer_Add_on_FFPE_outs.zarr")
he_image_path = os.path.join(xenium_dir, "Xenium_V1_hPancreas_Cancer_Add_on_FFPE_he_image.ome.tif")

# =============================================================================
# Data loading with caching
# =============================================================================
def load_data():
    """
    Load SpatialData and H&E image (with OME metadata) once and cache globally.
    
    Why use this function pattern?
    Answer: It implements a lazy loading pattern with caching. Data is loaded only when needed
    and stored for future use, preventing redundant loading operations.
    
    Returns:
        sdata: SpatialData object containing spatial transcriptomics data
        he_image: H&E image as ndarray (Height, Width, Channels)
        psx, psy: pixel size in microns (physical dimensions of each pixel)
    """
    global _cached_sdata, _cached_he_image, _cached_pixel_size
    _cached_sdata = globals().get("_cached_sdata", None)
    _cached_he_image = globals().get("_cached_he_image", None)
    _cached_pixel_size = globals().get("_cached_pixel_size", None)
    
    # Load SpatialData if not already cached
    if _cached_sdata is None:
        print("→ Loading SpatialData Zarr …")
        _cached_sdata = sd.read_zarr(zarr_path)  # Load data from Zarr storage format
        print("✔ SpatialData loaded.")
    else:
        print("← Reusing cached SpatialData.")

    # Load H&E image and metadata if not already cached
    if _cached_he_image is None or _cached_pixel_size is None:
        print("→ Loading H&E TIFF (memmap) & reading OME metadata …")
        
        # Why use memmap?
        # Answer: Memory mapping allows files to be accessed without loading the entire file into
        # memory, which is crucial for large image files that might exceed available RAM.
        with TiffFile(he_image_path) as tif:
            img = tif.asarray(out='memmap')  # Load image as memory-mapped array
            ome_xml = tif.ome_metadata  # Extract OME metadata
        
        # Why check and potentially transpose the image?
        # Answer: Standardizes the image dimensions to have channels as the last dimension (H,W,C),
        # regardless of how they were stored in the TIFF file (which might be C,H,W).
        if img.ndim == 3 and img.shape[0] in (1, 3):
            img = np.moveaxis(img, 0, -1)  # Convert from (C,H,W) to (H,W,C) format
        
        # Parse OME-XML metadata to extract pixel size information
        # Why use ElementTree and namespaces?
        # Answer: OME-TIFF follows a standardized XML schema with specific namespaces.
        # Using ElementTree with appropriate namespaces ensures correct parsing.
        root = ET.fromstring(ome_xml)
        ns = {'ns': 'http://www.openmicroscopy.org/Schemas/OME/2016-06'}
        pixels = root.find('.//ns:Pixels', ns)
        
        # Extract physical pixel size in microns
        # Why default to 1.0?
        # Answer: Ensures the code doesn't fail if the metadata is missing.
        # A value of 1.0 implies 1 pixel = 1 micron as a fallback.
        psx = float(pixels.attrib.get('PhysicalSizeX', 1.0))
        psy = float(pixels.attrib.get('PhysicalSizeY', 1.0))
        
        # Store loaded data in cache
        _cached_he_image = img
        _cached_pixel_size = (psx, psy)
        
        print(f"✔ H&E loaded (shape={img.shape}), pixel sizes: X={psx} µm, Y={psy} µm")
    else:
        print("← Reusing cached H&E image and pixel sizes.")
        img = _cached_he_image
        psx, psy = _cached_pixel_size

    return _cached_sdata, img, psx, psy

# -----------------------------------------------------------------------------
# QC & preprocessing functions
# -----------------------------------------------------------------------------
def paper_specific_qc(patches, qr):
    """
    Apply additional quality control specific to paper requirements.
    
    Why add paper-specific QC?
    Answer: This addresses issues specific to the analysis pipeline or publication
    requirements that weren't covered by the general QC. In this case, it removes
    completely black patches at the largest zoom level.
    
    Parameters:
        patches: Dictionary of patches per cell
        qr: Quality results
        
    Returns:
        Updated patches and quality results
    """
    # Identify completely black patches at kappa=2.0
    to_rm = []
    for cid in patches:
        for i,q in enumerate(qr[cid]):
            if q['kappa']==2.0 and patches[cid][i].mean()<5:
                to_rm.append((cid,i))
    
    print(f"Removing {len(to_rm)} completely black κ=2.0 patches")
    
    # Mark identified patches as invalid
    for cid,i in to_rm:
        qr[cid][i]['is_valid']=False
        qr[cid][i]['reason']="Completely black patch"
    
    return patches, qr

def mad_filter_cells(sdata, column="total_counts", verbose=True):
    """
    Filter cells based on MAD (Median Absolute Deviation) of a given column.
    
    Parameters:
        sdata: SpatialData object
        column: Column name in sdata.table.obs to apply MAD filtering on
        verbose: Whether to print status messages
    
    Returns:
        valid_cell_ids: Numpy array of filtered cell IDs
        threshold: Threshold value used for filtering
    """
    if verbose:
        print("→ Applying MAD-based filtering on", column)

    # Extract AnnData table
    cell_table = sdata.table if hasattr(sdata, 'table') else sdata['table']

    if column not in cell_table.obs.columns:
        raise ValueError(f"Missing '{column}' column in SpatialData table.")

    counts = cell_table.obs[column].values
    median_C = np.median(counts)
    mad_C = np.median(np.abs(counts - median_C))
    threshold = median_C - mad_C
    keep_mask = counts >= threshold

    valid_cell_table = cell_table[keep_mask]
    valid_cell_ids = valid_cell_table.obs_names.to_numpy()

    n_valid = keep_mask.sum()
    if n_valid == 0:
        raise ValueError("No cells passed MAD filtering. Consider adjusting the threshold.")

    if verbose:
        print(f"✔ {n_valid} cells kept (≥ {threshold:.1f} {column})")
    print(f"✔ {n_valid} cells kept (≥ {threshold:.1f} transcripts)")
    return valid_cell_ids, threshold

def extract_xenium_nucleus_centroids_from_spatialdata(sdata):
    """
    Extract nucleus centroids from a SpatialData object containing Xenium data.
    """
    print("[DEBUG] Entering extract_xenium_nucleus_centroids_from_spatialdata")
    print(f"[DEBUG] Available shape keys: {list(sdata.shapes.keys())}")

    # Try to get nucleus boundaries or cell circles to extract centroids
    if 'nucleus_boundaries' in sdata.shapes:
        nucleus_shapes = sdata.shapes['nucleus_boundaries']
        print(f"[DEBUG] nucleus_shapes contains {len(nucleus_shapes)} geometries")
        nucleus_centroids = np.array([
            (geom.centroid.x, geom.centroid.y) 
            for geom in nucleus_shapes.geometry
        ])
        cell_ids = nucleus_shapes.index.values
        print(f"[DEBUG] Extracted {nucleus_centroids.shape[0]} nucleus centroids")

    elif 'cell_circles' in sdata.shapes:
        print("[DEBUG] Using 'cell_circles' from sdata.shapes")
        cell_circles = sdata.shapes['cell_circles']
        print(f"[DEBUG] cell_circles contains {len(cell_circles)} geometries")
        nucleus_centroids = np.array([
            (geom.centroid.x, geom.centroid.y) 
            for geom in cell_circles.geometry
        ])
        cell_ids = cell_circles.index.values
        print(f"[DEBUG] Extracted {nucleus_centroids.shape[0]} cell circle centroids as nucleus centroids")

    else:
        print("[DEBUG] Neither 'nucleus_boundaries' nor 'cell_circles' found. Trying 'spatial' in obsm...")
        if 'spatial' in sdata.tables['table'].obsm:
            nucleus_centroids = sdata.tables['table'].obsm['spatial']
            cell_ids = sdata.tables['table'].obs_names.values
            print(f"[DEBUG] Extracted {nucleus_centroids.shape[0]} centroids from obsm['spatial']")
        else:
            raise ValueError("Could not find nucleus centroids in the SpatialData object")

    print("[DEBUG] Exiting extract_xenium_nucleus_centroids_from_spatialdata")
    return nucleus_centroids, cell_ids

def sample_grid_cells(centroids, grid_size=8, cells_per_grid=5):
    """
    Sample cells from a spatial grid for balanced representation.
    
    Why use grid sampling?
    Answer: Random sampling might oversample dense regions and undersample sparse ones. -> some regions may have many detected cells (high density), while others are sparse
    If you randomly sample cells, you'll likely pick many from dense regions and miss spatially interesting areas.
    Grid sampling samples cells evenly across space using a grid.
    
    Parameters:
        centroids: Array of (x,y) coordinates
        grid_size: Number of grid divisions in each dimension -> Default = 8 → 8×8 = 64 grid cells.
        cells_per_grid: Maximum cells to sample from each grid cell -> Default = 5
        
    Returns:
        Array of indices of sampled cells
    """
    xs, ys = centroids[:,0], centroids[:,1]
    xmin,xmax, ymin,ymax = xs.min(), xs.max(), ys.min(), ys.max()
    
    # Calculate grid cell sizes
    dx, dy = (xmax-xmin)/grid_size, (ymax-ymin)/grid_size
    
    out = []
    # Iterate through each grid cell
    for i in range(grid_size):
        for j in range(grid_size):
            # Find cells within current grid cell
            m = ((xs>=xmin+i*dx)&(xs<xmin+(i+1)*dx)&
                 (ys>=ymin+j*dy)&(ys<ymin+(j+1)*dy))
            idx = np.where(m)[0]
            
            # Sample cells from current grid cell
            if idx.size:
                take = min(cells_per_grid, idx.size)
                out.extend(np.random.choice(idx, take, replace=False))
    
    return np.array(out)

# =============================================================================
# Fast downsample preview via strided slicing
# =============================================================================
def create_downsampled_preview(he_image, coords_px, sample_indices, factor=10):
    """
    Create a downsampled preview of the H&E image with sampled cell locations.
    
    Why create a downsampled preview?
    Answer: Full-resolution H&E images can be extremely large (gigapixels). A downsampled
    preview allows quick visual verification of coordinate mapping and sampling without
    requiring excessive memory or computation.
    
    Parameters:
        he_image: Full H&E image
        coords_px: Pixel coordinates of all cells
        sample_indices: Indices of sampled cells
        factor: Downsampling factor
    """
    h, w = he_image.shape[:2]
    small_h, small_w = h // factor, w // factor
    print(f"→ Fast downsample to ({small_h}, {small_w}) via slicing …")
    
    # Downsample image via strided slicing
    # Why use slicing instead of interpolation?
    # Answer: Much faster for large images, and sufficient for preview purposes
    small_image = he_image[::factor, ::factor, :].astype(np.uint8)
    
    # Scale down coordinates to match downsampled image
    pts = coords_px[sample_indices] // factor
    x_pts, y_pts = pts[:, 0], pts[:, 1]
    
    # Visualize sampled points on downsampled image
    plt.figure(figsize=(8, 8))
    plt.imshow(small_image)
    plt.scatter(x_pts, y_pts, c='red', s=10, alpha=0.7)
    
    # Label first 20 points for reference
    for i, idx in enumerate(sample_indices[:20]):
        x, y = coords_px[idx] // factor
        plt.text(x + 5, y + 5, str(i), color='white', fontsize=9,
                 bbox=dict(facecolor='black', alpha=0.7))
    
    plt.axis('off')
    plt.tight_layout()
    plt.show()


from skimage.io import imread
from skimage.transform import estimate_transform, AffineTransform
import napari

def align_xenium_to_he(he_image, xenium_coords):
    """
    Manually pick landmarks using Napari to align Xenium coordinates to the H&E image.

    Parameters:
        he_image (ndarray): H&E image (H, W) or (H, W, C)
        xenium_coords (ndarray): Xenium centroid coordinates (N, 2)

    Returns:
        aligned_coords (ndarray): Transformed Xenium coordinates aligned to H&E space (N, 2)
        transform_matrix (ndarray): 3x3 affine transformation matrix
    """

    # Open napari viewer
    viewer = napari.Viewer()
    viewer.add_image(he_image, name="H&E Image")
    viewer.add_points(xenium_coords, name="Xenium Coordinates", size=10, face_color="red")

    # Create empty layers for user to pick landmarks
    he_points_layer = viewer.add_points(name="H&E Landmarks", size=10, face_color="blue")
    xenium_points_layer = viewer.add_points(name="Xenium Landmarks", size=10, face_color="green")

    print("\n💡 INSTRUCTIONS:")
    print("- Use the 'H&E Landmarks' layer to click landmarks on the H&E image.")
    print("- Use the 'Xenium Landmarks' layer to click corresponding points in Xenium space.")
    print("- Pick the same number of points in the same order.")
    print("- Close the napari window to continue.\n")

    napari.run()

    # Retrieve landmarks
    he_landmarks = he_points_layer.data
    xenium_landmarks = xenium_points_layer.data

    if len(he_landmarks) != len(xenium_landmarks):
        raise ValueError(f"❌ Mismatched landmarks: {len(he_landmarks)} (H&E) vs {len(xenium_landmarks)} (Xenium)")

    # Estimate affine transformation
    transform = estimate_transform('affine', src=xenium_landmarks, dst=he_landmarks)

    # Apply to all Xenium coordinates
    aligned_coords = transform(xenium_coords)

    print("✅ Transformation complete. Xenium coordinates aligned to H&E image.")

    return aligned_coords, transform.params

def apply_saved_transform(transform_path, new_coords_path, output_path):
    """
    Apply a saved affine transformation to new Xenium coordinates.

    Parameters:
        transform_path (str): Path to the saved transformation matrix (CSV).
        new_coords_path (str): Path to the new Xenium coordinates (CSV).
        output_path (str): Path to save the aligned coordinates (CSV).

    Returns:
        None
    """
    # Load saved transform matrix
    loaded_matrix = np.loadtxt(transform_path, delimiter=",")
    transform = AffineTransform(matrix=loaded_matrix)

    # Load new Xenium coordinates
    new_coords = np.loadtxt(new_coords_path, delimiter=",")

    # Apply the transformation
    aligned_new_coords = transform(new_coords)

    # Save the aligned coordinates
    np.savetxt(output_path, aligned_new_coords, delimiter=",")
    print(f"Aligned coordinates saved to {output_path}")
    
# =============================================================================
# Main pipeline with optional preloading
# =============================================================================
# Import the alignment functions
from xenium_he_aligner import align_xenium_to_he, apply_transform

def main_patch_extraction_pipeline(sdata=None, he_image=None, psx=None, psy=None):
    """
    Main function that orchestrates the entire patch extraction pipeline.
    
    Why have a comprehensive pipeline function?
    Answer: Organizes the entire workflow in a logical sequence, making it easier to:
    1. Understand the complete process
    2. Identify bottlenecks
    3. Reuse the pipeline with different datasets
    4. Optionally skip steps by providing preloaded data
    
    Parameters:
        sdata: Optional preloaded SpatialData object
        he_image: Optional preloaded H&E image
        psx, psy: Optional preloaded pixel sizes
        
    Returns:
        Dictionary containing all extracted data and results
    """
    t_start = time.time()
    
    # Load data if not provided
    if sdata is None or he_image is None or psx is None or psy is None:
        sdata, he_image, psx, psy = load_data()
    else:
        print("← Using provided SpatialData and H&E image, skipping load_data.")
    
    # ------------------------------------------------------------------------
    # Step 1: Filter cells based on total counts using MAD
    # ------------------------------------------------------------------------
    print("→ Filtering cells based on total counts using MAD …")
    valid_cell_ids, threshold = mad_filter_cells(sdata)

    # ------------------------------------------------------------------------
    # Step 2: Extract centroids of valid cells from SpatialData geometry
    # ------------------------------------------------------------------------
    print("→ Extracting centroids from SpatialData geometry …")

    # Use helper function to get all centroids
    all_centroids_um, all_cell_ids = extract_xenium_nucleus_centroids_from_spatialdata(sdata)
    """
    # Print debug information about cell ID formats 
    print(f"→ Debug: First few valid_cell_ids: {valid_cell_ids[:3]}")
    print(f"→ Debug: First few all_cell_ids: {all_cell_ids[:3]}")
    print(f"→ Debug: Types - valid_cell_ids: {type(valid_cell_ids[0])}, all_cell_ids: {type(all_cell_ids[0])}")

    # Debug: Print lengths of valid and all cell IDs
    print(f"→ Debug: Number of valid_cell_ids: {len(valid_cell_ids)}")
    print(f"→ Debug: Number of all_cell_ids: {len(all_cell_ids)}")

    # Ensure consistent ID format (convert both to lowercase strings and strip whitespace)
    valid_cell_ids_set = set(str(cid).strip().lower() for cid in valid_cell_ids)
    all_cell_ids_str = np.array([str(cid).strip().lower() for cid in all_cell_ids])

    # Debug: Print a few IDs after formatting
    print(f"→ Debug: First few formatted valid_cell_ids: {list(valid_cell_ids_set)[:3]}")
    print(f"→ Debug: First few formatted all_cell_ids: {all_cell_ids_str[:3]}")

    # Filter centroids to match valid cell IDs
    keep_mask = np.array([cid in valid_cell_ids_set for cid in all_cell_ids_str])
    centroids_um = all_centroids_um[keep_mask]
    cell_ids = np.array([all_cell_ids[i] for i in np.where(keep_mask)[0]])

    # Debug: Print the number of matched cells
    print(f"→ Debug: Number of matched cells: {len(centroids_um)}")

    # Check if we have matching cells
    if len(centroids_um) == 0:
        # Debug: Print unmatched IDs for further investigation
        unmatched_ids = set(all_cell_ids_str) - valid_cell_ids_set
        print(f"→ Debug: Unmatched IDs (sample): {list(unmatched_ids)[:10]}")
        raise ValueError("No cells matched between filtered AnnData and nucleus boundaries. Check cell ID formats.")
    """
    print(f"✔ Extracted {len(centroids_um)} valid centroids after MAD filtering\n")
    
    # ------------------------------------------------------------------------
    # Step 3: Align Xenium coordinates to H&E image
    # ------------------------------------------------------------------------
    # Use the updated alignment function that checks for existing transformation
    print("→ Starting alignment process…")
    coords_px, affine = align_xenium_to_he(
        sdata=sdata,
        he_image=he_image,
        centroids_um=centroids_um,
        psx=psx,
        psy=psy,
        xenium_dir=xenium_dir  # Make sure xenium_dir is defined or passed as a parameter
    )
    print("✅ Alignment completed successfully\n")

    # ------------------------------------------------------------------------

    # Convert physical coordinates (microns) to pixel coordinates
    print("→ Converting centroids from µm → pixels …")
    coords_px = np.zeros_like(centroids_um, dtype=int)
    coords_px[:, 0] = np.round(centroids_um[:, 0] / psx).astype(int)
    coords_px[:, 1] = np.round(centroids_um[:, 1] / psy).astype(int)
    print("✔ Sample pixel coords:", coords_px[:5], "\n")
    
    # Clip coordinates to image boundaries
    # Why clip coordinates?
    # Answer: Prevents out-of-bounds errors when extracting patches.
    # Some cells might be detected slightly outside the imaged area.
    h, w = he_image.shape[:2]
    coords_px[:, 0] = np.clip(coords_px[:, 0], 0, w - 1)
    coords_px[:, 1] = np.clip(coords_px[:, 1], 0, h - 1)
    
    # Sample cells using grid-based approach
    # Why sample instead of using all cells?
    # Answer: Processing all cells would be computationally expensive and unnecessary
    # for quality assessment or method development. Grid sampling ensures spatial
    # diversity while keeping the sample size manageable.
    sample_idx = sample_grid_cells(coords_px, grid_size=8, cells_per_grid=5)
    print(f"→ Sampled {len(sample_idx)} cells\n")
    
    # Create visual preview of sampled cells
    create_downsampled_preview(he_image, coords_px, sample_idx)
    
    # Extract patches only for sampled cells and assess quality
    # Extract and enhance patches without quality assessment
    print("→ Extracting patches for sampled cells …")
    test_idxs = sample_idx[:min(50, len(sample_idx))]  # Limit for testing/visualization
    enhanced_patches, normalized_patches = extract_image_patches_enhanced(
        he_image, coords_px[test_idxs]
    )

    # Create dummy quality results: all valid initially
    quality_results = {}
    kappas = [0.5, 0.75, 1.0, 1.5, 2.0]
    for cid in enhanced_patches:
        quality_results[cid] = [
            {'kappa': k, 'score': 1.0, 'is_valid': True, 'reason': 'Valid'}
            for k in kappas
        ]

    # Apply black-patch QC at kappa=2.0 only
    final_patches, final_quality = paper_specific_qc(enhanced_patches, quality_results)

    # Skip QC summary, distribution plots, etc.

    # (Optional) Just show a few patches without filtering by score
    print(f"✔ Extracted patches for {len(final_patches)} cells\n")
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    axes = axes.flatten()
    for i, (cid, patches_list) in enumerate(list(final_patches.items())[:10]):
        patch = patches_list[-1]  # Show κ=2.0 as an example
        axes[i].imshow(patch)
        axes[i].axis('off')
        axes[i].set_title(f"Cell {cid}\nκ=2.0")
    plt.tight_layout()
    plt.show()

    
    # Calculate and display total processing time
    print(f"\nTotal processing time: {time.time() - t_start:.1f}s")
    
    # Return all extracted data for potential downstream use
    # Why return a comprehensive dictionary?
    # Answer: Allows flexibility for different downstream analyses,
    # debugging, or further processing without rerunning the entire pipeline.
    return {
        'centroids_px': coords_px,
        'cell_ids': cell_ids,
        'sample_indices': sample_idx,
        'enhanced_patches': final_patches,
        'normalized_patches': normalized_patches,
        'quality_results': final_quality,
    }

if __name__ == '__main__':
    # Optionally preload data once
    sdata_cache, he_image_cache, psx_cache, psy_cache = load_data()
    # Then reuse caches on subsequent calls
    results = main_patch_extraction_pipeline(
        sdata=sdata_cache,
        he_image=he_image_cache,
        psx=psx_cache,
        psy=psy_cache
    )

← Reusing cached SpatialData.
← Reusing cached H&E image and pixel sizes.
← Using provided SpatialData and H&E image, skipping load_data.
→ Filtering cells based on total counts using MAD …
→ Applying MAD-based filtering on total_counts
✔ 147707 cells kept (≥ 67.0 total_counts)
✔ 147707 cells kept (≥ 67.0 transcripts)
→ Extracting centroids from SpatialData geometry …
[DEBUG] Entering extract_xenium_nucleus_centroids_from_spatialdata
[DEBUG] Available shape keys: ['cell_boundaries', 'cell_circles', 'nucleus_boundaries']
[DEBUG] nucleus_shapes contains 190965 geometries


/var/folders/c0/44p3b4xx4v956042lj_vp7z80000gn/T/ipykernel_69764/4117260420.py:184: DeprecationWarning: Table accessor will be deprecated with SpatialData version 0.1, use sdata.tables instead.
  cell_table = sdata.table if hasattr(sdata, 'table') else sdata['table']


[DEBUG] Extracted 190965 nucleus centroids
[DEBUG] Exiting extract_xenium_nucleus_centroids_from_spatialdata


NameError: name 'centroids_um' is not defined